In [2]:
import pandas as pd
import re
import os

In [3]:


def preprocess_string(string : str) -> str:
    string = string.lower().strip()
    string = re.sub(' +', ' ', string)
    string = string.replace("_", " ")
    string = string.replace("-", " ")
    string = string.replace("&", " ")
    string = string.split("(")[0]
    string = string.split("#")[0]

    string = string.strip()

    # handle known synoynms
    synonyms = {
        "refrigerator": "fridge",
        "vaccumcleaner": "vacuum cleaner",
        "breadmaker": "bread maker",
      
        
    }
    if "freezer" in string:
        string = "fridge"

    if string in synonyms:
        string = synonyms[string]

    if 'hi fi' in string:
        string = "audio system"

    if "router" in string:
        string = "router"

    if "treadmill" in string:
        string = "running machine"
        

    if "laptop" in string:
        string = "laptop"
    
    if "server" in string:
        string = "server"

    if "monitor" in string and not "baby" in string:
        string = "monitor"
    # special cases
    if "computer" in string and "charger" not in string:
        string = "pc"

    if "tv" in string:
        string = "television"

    if "television" in string:
        string = "television"

    if "macbook" in string:
        string = "laptop"
        
    if "car charger" == string:
        string = "ev"
    
    if "toast" in string:
        string = "toaster"
    
    if "modem" in string:
        string = "router"

    # we treat all audio devices as speakers so subwoofer is also a speaker
    if "subwoofer" in string:
        string = "speaker"

    if "speaker" in string:
        string = "speaker"

    if "iron" in string and "soldering" not in string:
        string = "iron"

    
    if "coffeemachine" in string:
        string = "coffee machine"
    if "coffee maker" in string:
        string = "coffee machine"

    if "dishwasher" in string:
        string = "dish washer"
    if "air conditioner" in string:
        string = "ac"

    if "air conditioning" in string:
        string = "ac"
    
    string = re.sub(' +', ' ', string)
    string = re.sub(r'\d+', '', string)
    return string.strip()


In [4]:
labels = set()
for f in os.listdir("./Energy_graph/data/processed_watts/"):
    if f.endswith(".pkl"):
        data = pd.read_pickle("./Energy_graph/data/processed_watts/" + f)
        for h in data:
            for k in data[h]:
                if "aggregate" in k:
                    continue
                labels.add(preprocess_string(k))


labels = list(labels)


len(labels)

64

In [6]:

labels

['fridge',
 'food mixer',
 'speaker',
 'tablet',
 'gaming pc',
 'watercooker',
 'stereo',
 'solar thermal pumping station',
 'laptop',
 'oven',
 'router',
 'soldering iron',
 'electric furnace',
 'tumble dryer',
 'kimchi fridge',
 'vivarium',
 'waste disposal unit',
 'air handling unit',
 'htpc',
 'television',
 'fountain',
 'fan',
 'oven extractor fan',
 'games console',
 'server',
 'hair straighteners',
 'audio system',
 'electric heating element',
 'air exchanger',
 'whirlpool bath',
 'blender',
 'electric stove',
 'toaster',
 'external hard disk',
 'ev',
 'air exhaust',
 'microwave',
 'printer',
 'stove',
 'network attached storage',
 'cooker',
 'bouncy castle pump',
 'audio amplifier',
 'minioven',
 'smoke alarm',
 'dryer',
 'dish washer',
 'coffee machine',
 'boiler',
 'pc',
 'food processor',
 'kettle',
 'water purifier',
 'iron',
 'hair dryer',
 'monitor',
 'washing machine',
 'electric heater',
 'vacuum cleaner',
 'pond pump',
 'washer dryer',
 'electric oven',
 'bread maker',

In [ ]:
import pickle 
with open("./Energy_graph/data/labels_new.pkl", "wb") as f:
    pickle.dump(labels, f)

In [16]:
for f in os.listdir("./Energy_graph/data/processed_watts/"):
    if f.endswith(".pkl"):
        data = pd.read_pickle("./Energy_graph/data/processed_watts/" + f)
        for h in data:
            # print(data[h]["aggregate"].max())
            if int(data[h]["aggregate"].max()) < 1000:
                print(data[h]["aggregate"].max(), h)
                # continue                


# WordNet

In [1]:
%pip install nltk

import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached regex-2023.10.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (773 kB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import os 
import re
def preprocess_string_basic(string : str) -> str:
    string = string.lower().strip()
    string = re.sub(' +', ' ', string)
    string = string.replace("_", " ")
    string = string.replace("-", " ")
    string = string.replace("&", " ")
    string = string.split("(")[0]
    string = string.split("#")[0]

    string = string.strip()

    string = re.sub(' +', ' ', string)
    string = re.sub(r'\d+', '', string)
    return string.strip()



def group_devices(devices):
    device_groups = {}
    for device in devices:
        synsets = wn.synsets(device)
        if synsets:
            hypernyms = synsets[0].hypernyms()
            if hypernyms:
                hypernym_name = hypernyms[0].lemmas()[0].name()
                device_groups.setdefault(hypernym_name, []).append(device)
    return device_groups

labels = set()
for f in os.listdir("./Energy_graph/data/processed_watts/"):
    if f.endswith(".pkl"):
        data = pd.read_pickle("./Energy_graph/data/processed_watts/" + f)
        for h in data:
            for k in data[h]:
                if "aggregate" in k:
                    continue
                labels.add(preprocess_string_basic(k))

devices = list(labels)
grouped_devices = group_devices(devices)
grouped_devices

{'vessel': ['boiler'],
 'white_goods': ['dishwasher', 'refrigerator'],
 'kitchen_appliance': ['stove', 'oven'],
 'broadcasting': ['tv', 'television'],
 'digital_computer': ['pc'],
 'exercise_device': ['treadmill'],
 'pot': ['kettle'],
 'metallic_element': ['iron'],
 'portable_computer': ['laptops', 'laptop'],
 'optical_device': ['projector'],
 'supervisor': ['monitor'],
 'electronic_equipment': ['modem'],
 'refrigerator': ['fridge'],
 'structure': ['fountain'],
 'cooking_utensil': ['cooker'],
 'device': ['heater', 'fan'],
 'electromagnetic_radiation': ['microwave'],
 'mixer': ['blender'],
 'electric_refrigerator': ['freezer'],
 'skilled_worker': ['printer', 'router'],
 'machine': ['computer'],
 'reproducer': ['stereo'],
 'appliance': ['dryer'],
 'supporter': ['toaster']}